In [7]:
import os
import html2text
import json
import re
from collections import defaultdict, Counter
from datetime import datetime

def html_to_markdown(html_content):
    """
    将 HTML 内容转换为 Markdown 格式
    """
    converter = html2text.HTML2Text()
    converter.ignore_links = False       # 保留链接
    converter.ignore_images = False      # 保留图片
    converter.body_width = 0             # 不自动换行
    converter.single_line_break = True   # 单换行符换行

    return converter.handle(html_content)

def extract_triples_from_json(json_file):
    """从JSON文件加载三元组数据"""
    try:
        with open(json_file, 'r', encoding='utf-8') as f:
            return json.load(f)
    except Exception as e:
        print(f"❌ 加载JSON文件失败: {e}")
        return None

def analyze_graph_structure(triples):
    """分析图谱结构并生成统计信息"""
    if not triples:
        return {}
    
    # 基本统计
    total_triples = len(triples)
    entities = set()
    predicates = []
    inferred_count = 0
    
    for triple in triples:
        entities.add(triple.get('subject', ''))
        entities.add(triple.get('object', ''))
        predicates.append(triple.get('predicate', ''))
        if triple.get('inferred', False):
            inferred_count += 1
    
    # 谓词统计
    predicate_counts = Counter(predicates)
    
    # 实体连接度统计
    entity_connections = defaultdict(int)
    for triple in triples:
        entity_connections[triple.get('subject', '')] += 1
        entity_connections[triple.get('object', '')] += 1
    
    return {
        'total_triples': total_triples,
        'total_entities': len(entities),
        'total_predicates': len(set(predicates)),
        'inferred_triples': inferred_count,
        'original_triples': total_triples - inferred_count,
        'predicate_counts': predicate_counts,
        'entity_connections': entity_connections,
        'top_entities': sorted(entity_connections.items(), key=lambda x: x[1], reverse=True)[:10]
    }

def generate_knowledge_graph_report(triples, title="知识图谱分析报告"):
    """生成知识图谱专门的Markdown报告"""
    if not triples:
        return "❌ 没有找到有效的三元组数据"
    
    # 分析图谱结构
    stats = analyze_graph_structure(triples)
    
    # 生成Markdown内容
    markdown_content = []
    
    # 标题和基本信息
    markdown_content.append(f"# {title}")
    markdown_content.append(f"\n📅 **生成时间**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    markdown_content.append(f"\n---\n")
    
    # 图谱统计
    markdown_content.append("## 📊 图谱统计")
    markdown_content.append(f"- **总实体数**: {stats['total_entities']}")
    markdown_content.append(f"- **总关系数**: {stats['total_triples']}")
    markdown_content.append(f"- **原始关系**: {stats['original_triples']}")
    markdown_content.append(f"- **推理关系**: {stats['inferred_triples']}")
    markdown_content.append(f"- **谓词类型**: {stats['total_predicates']}")
    
    # 重要实体
    markdown_content.append(f"\n## 🔗 重要实体 (按连接度排序)")
    markdown_content.append("| 实体 | 连接数 |")
    markdown_content.append("|------|--------|")
    for entity, count in stats['top_entities']:
        markdown_content.append(f"| {entity} | {count} |")
    
    # 常见关系类型
    markdown_content.append(f"\n## 🔄 常见关系类型")
    markdown_content.append("| 关系类型 | 出现次数 |")
    markdown_content.append("|----------|----------|")
    for predicate, count in stats['predicate_counts'].most_common(10):
        markdown_content.append(f"| {predicate} | {count} |")
    
    # 按类型分组显示三元组
    original_triples = [t for t in triples if not t.get('inferred', False)]
    inferred_triples = [t for t in triples if t.get('inferred', False)]
    
    if original_triples:
        markdown_content.append(f"\n## 📋 原始关系 ({len(original_triples)} 个)")
        markdown_content.append("| 主语 | 谓语 | 宾语 |")
        markdown_content.append("|------|------|------|")
        for triple in sorted(original_triples, key=lambda x: x.get('subject', ''))[:20]:  # 只显示前20个
            subject = triple.get('subject', '').replace('|', '\\|')
            predicate = triple.get('predicate', '').replace('|', '\\|')
            obj = triple.get('object', '').replace('|', '\\|')
            markdown_content.append(f"| {subject} | {predicate} | {obj} |")
        
        if len(original_triples) > 20:
            markdown_content.append(f"\n*... 还有 {len(original_triples) - 20} 个原始关系*")
    
    if inferred_triples:
        markdown_content.append(f"\n## 🧠 推理关系 ({len(inferred_triples)} 个)")
        markdown_content.append("| 主语 | 谓语 | 宾语 |")
        markdown_content.append("|------|------|------|")
        for triple in sorted(inferred_triples, key=lambda x: x.get('subject', ''))[:15]:  # 只显示前15个
            subject = triple.get('subject', '').replace('|', '\\|')
            predicate = triple.get('predicate', '').replace('|', '\\|')
            obj = triple.get('object', '').replace('|', '\\|')
            markdown_content.append(f"| {subject} | {predicate} | {obj} |")
        
        if len(inferred_triples) > 15:
            markdown_content.append(f"\n*... 还有 {len(inferred_triples) - 15} 个推理关系*")
    
    # 生成实体网络图（简化版）
    markdown_content.append(f"\n## 🌐 核心实体关系网络")
    markdown_content.append("```")
    markdown_content.append("主要实体及其直接关系:")
    
    # 选择前5个最重要的实体
    for entity, _ in stats['top_entities'][:5]:
        markdown_content.append(f"\n🔸 {entity}:")
        related_triples = [t for t in triples if t.get('subject') == entity or t.get('object') == entity]
        for triple in related_triples[:5]:  # 只显示前5个关系
            if triple.get('subject') == entity:
                markdown_content.append(f"  → {triple.get('predicate')} → {triple.get('object')}")
            else:
                markdown_content.append(f"  ← {triple.get('predicate')} ← {triple.get('subject')}")
    
    markdown_content.append("```")
    
    # 生成Mermaid图
    markdown_content.append(f"\n## 🎯 关系图谱 (Mermaid)")
    markdown_content.append("```mermaid")
    markdown_content.append("graph TD")
    
    # 选择重要的关系生成Mermaid图
    node_map = {}
    node_counter = 0
    
    # 为重要实体创建节点
    for entity, _ in stats['top_entities'][:8]:  # 只显示前8个重要实体
        node_id = f"N{node_counter}"
        node_map[entity] = node_id
        # 清理实体名称中的特殊字符
        clean_entity = entity.replace('"', '').replace("'", "")
        markdown_content.append(f'    {node_id}["{clean_entity}"]')
        node_counter += 1
    
    # 添加关系
    added_edges = set()
    for triple in triples:
        subject = triple.get('subject', '')
        obj = triple.get('object', '')
        predicate = triple.get('predicate', '')
        
        if subject in node_map and obj in node_map:
            edge_key = (subject, obj, predicate)
            if edge_key not in added_edges:
                subject_node = node_map[subject]
                obj_node = node_map[obj]
                # 清理谓词名称
                clean_predicate = predicate.replace('"', '').replace("'", "")
                markdown_content.append(f'    {subject_node} -->|"{clean_predicate}"| {obj_node}')
                added_edges.add(edge_key)
    
    markdown_content.append("```")
    
    return "\n".join(markdown_content)

def convert_html_file_to_knowledge_graph_markdown(input_html_path, output_md_path=None):
    """
    读取知识图谱HTML文件并转换为专门的知识图谱Markdown报告
    """
    if not os.path.exists(input_html_path):
        print(f"❌ 文件不存在: {input_html_path}")
        return

    if output_md_path is None:
        base_name = os.path.splitext(input_html_path)[0]
        output_md_path = f"{base_name}_知识图谱报告.md"

    # 首先尝试从对应的JSON文件加载结构化数据
    json_file = input_html_path.replace('.html', '.json')
    triples = None
    
    if os.path.exists(json_file):
        print(f"📄 找到对应的JSON文件: {json_file}")
        triples = extract_triples_from_json(json_file)
        
        if triples:
            print(f"✅ 成功从JSON文件加载了 {len(triples)} 个三元组")
            # 从文件名提取标题
            title = os.path.basename(input_html_path).replace('.html', '') + " - 知识图谱分析报告"
            
            # 生成专门的知识图谱报告
            markdown_content = generate_knowledge_graph_report(triples, title)
            
            # 保存报告
            with open(output_md_path, 'w', encoding='utf-8') as f:
                f.write(markdown_content)
            
            print(f"✅ 知识图谱报告已保存为: {output_md_path}")
            return
    
    # 如果没有JSON文件，则使用传统的HTML转换方法
    print("⚠️ 未找到对应的JSON文件，使用HTML直接转换...")
    
    with open(input_html_path, 'r', encoding='utf-8') as f:
        html_content = f.read()

    markdown_content = html_to_markdown(html_content)

    with open(output_md_path, 'w', encoding='utf-8') as f:
        f.write(markdown_content)

    print(f"✅ 已成功转换并保存为: {output_md_path}")

# 使用示例
input_html = "/Users/liangzijun/Documents/2025金种子/knowledege_graph/ai-knowledge-graph/道路施工许可流程图谱.html"
output_md = "/Users/liangzijun/Documents/2025金种子/knowledege_graph/ai-knowledge-graph/道路施工许可流程图谱.md"

convert_html_file_to_knowledge_graph_markdown(input_html, output_md)



📄 找到对应的JSON文件: /Users/liangzijun/Documents/2025金种子/knowledege_graph/ai-knowledge-graph/道路施工许可流程图谱.json
✅ 成功从JSON文件加载了 88 个三元组
✅ 知识图谱报告已保存为: /Users/liangzijun/Documents/2025金种子/knowledege_graph/ai-knowledge-graph/道路施工许可流程图谱.md
